In [1]:
import warnings
warnings.filterwarnings('ignore')
import hoomd
import gsd
import matplotlib.pyplot as plt
import numpy as np
import gsd.hoomd
from flowermd.base import Pack,Lattice, Simulation
from flowermd.library import EllipsoidForcefield, EllipsoidChain
from flowermd.utils import get_target_box_number_density
from flowermd.utils.constraints import create_rigid_ellipsoid_chain
import unyt as u
import hoomd

In [18]:
ellipsoid_chain = EllipsoidChain(lengths=1,num_mols=100,lpar=1.0,bead_mass=1.0)
ff = EllipsoidForcefield(epsilon=1.0,lpar=1.0,lperp=0.5,r_cut=2.5)
ff.hoomd_forces
system = Pack(molecules=ellipsoid_chain, density=0.1*u.Unit("nm**-3"), packing_expand_factor=6,edge=2,overlap=1,fix_orientation=True)
#system = Lattice(molecules=ellipsoid_chain,x=1,y=1,n=4)
gsd_path=('ellipsoid-chain-2mer.gsd')
rigid_frame, rigid = create_rigid_ellipsoid_chain(
    system.hoomd_snapshot
)
ellipsoid_sim = Simulation(
    initial_state=rigid_frame,
    forcefield=ff.hoomd_forces,
    constraint=rigid,
    dt=0.001,
    gsd_write_freq=int(1e4),
    gsd_file_name=gsd_path,
    log_write_freq=int(1e4),
    log_file_name='log.txt')

target_box = get_target_box_number_density(density=0.1*u.Unit("nm**-3"),n_beads=100)
ellipsoid_sim.run_update_volume(final_box_lengths=target_box, kT=6.0, n_steps=1e5,tau_kt=100*ellipsoid_sim.dt,period=10,thermalize_particles=True)
print("shrink finished")
ellipsoid_sim.run_NVT(n_steps=1e5, kT=1.0, tau_kt=10*ellipsoid_sim.dt)
#ellipsoid_sim.save_restart_gsd("restart.gsd")
ellipsoid_sim.flush_writers()
#ellipsoid_sim.save_simulation("sim.pickle")

Initializing simulation state from a gsd.hoomd.Frame.
Step 10000 of 100000; TPS: 14274.85; ETA: 0.1 minutes
Step 20000 of 100000; TPS: 15936.03; ETA: 0.1 minutes
Step 30000 of 100000; TPS: 16635.35; ETA: 0.1 minutes
Step 40000 of 100000; TPS: 16970.88; ETA: 0.1 minutes
Step 50000 of 100000; TPS: 17121.55; ETA: 0.0 minutes
Step 60000 of 100000; TPS: 16995.91; ETA: 0.0 minutes
Step 70000 of 100000; TPS: 16688.87; ETA: 0.0 minutes
Step 80000 of 100000; TPS: 16048.4; ETA: 0.0 minutes
Step 90000 of 100000; TPS: 15089.85; ETA: 0.0 minutes
Step 100000 of 100000; TPS: 12548.5; ETA: 0.0 minutes
shrink finished
Step 9999 of 100000; TPS: 2923.1; ETA: 0.5 minutes
Step 19999 of 100000; TPS: 2870.1; ETA: 0.5 minutes
Step 29999 of 100000; TPS: 3003.3; ETA: 0.4 minutes
Step 39999 of 100000; TPS: 3114.54; ETA: 0.3 minutes
Step 49999 of 100000; TPS: 3187.05; ETA: 0.3 minutes
Step 59999 of 100000; TPS: 3237.17; ETA: 0.2 minutes
Step 69999 of 100000; TPS: 3273.84; ETA: 0.2 minutes
Step 79999 of 100000; TP

In [ ]:
def ellipsoid_gsd(gsd_file, new_file, ellipsoid_types, lpar, lperp):
    """Add needed information to GSD file to visualize ellipsoids.

    Saves a new GSD file with lpar and lperp values populated
    for each particle. Ovito can be used to visualize the new GSD file.

    Parameters
    ----------
    gsd_file : str
        Path to the original GSD file containing trajectory information
    new_file : str
        Path and filename of the new GSD file
    ellipsoid_types : str or list of str
        The particle types (i.e. names) of particles to be drawn
        as ellipsoids.
    lpar : float
        Value of lpar of the ellipsoids
    lperp : float
        Value of lperp of the ellipsoids

    """
    with gsd.hoomd.open(new_file, "w") as new_t:
        with gsd.hoomd.open(gsd_file) as old_t:
            for snap in old_t:
                shape_dicts_list = []
                for ptype in snap.particles.types:
                    if ptype == ellipsoid_types or ptype in ellipsoid_types:
                        shapes_dict = {
                            "type": "Ellipsoid",
                            "a": lpar,
                            "b": lperp,
                            "c": lperp,
                        }
                    else:
                        shapes_dict = {"type": "Sphere", "diameter": 0.001}
                    shape_dicts_list.append(shapes_dict)
                snap.particles.type_shapes = shape_dicts_list
                snap.validate()
                new_t.append(snap)

gsd_path=('ellipsoid-chain-2mer.gsd')
ellipsoid_gsd(gsd_file=gsd_path,new_file="ovito-ellipsoid.gsd",ellipsoid_types='R',lpar=1.0,lperp=0.5)